In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def simulate_path_milstein(s0, mu, sigma, horizon, timesteps, n_sims, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    dt = horizon / timesteps  # Time step
    S = np.zeros((timesteps, n_sims))  # Initialize stock price matrix
    S[0, :] = s0  # Set initial stock price
    
    for i in range(1, timesteps):
        w = np.random.standard_normal(n_sims)  # Standard normal random variable
        dW = np.sqrt(dt) * w  # Brownian motion increment
        
        # Milstein correction term
        correction = 0.5 * sigma**2 * S[i-1, :] * (dW**2 - dt)
        
        # Apply the Milstein update
        S[i, :] = S[i-1, :] + mu * S[i-1, :] * dt + sigma * S[i-1, :] * dW + correction
    
    return S
def simulate_acdc(S0, r, sigma, T, N_paths, N_periods, days_per_year, m, K, B, G, contract='accumulator', seed=None):
    n_steps = int(days_per_year * T)
    #S = simulate_path(S0, r, sigma, T, n_steps, N_paths, seed)
    S = simulate_path_milstein(S0, r, sigma, T, n_steps, N_paths, seed)
    delta = 1 if contract.lower() == 'accumulator' else -1
    period_indices = np.linspace(0, n_steps, N_periods + 1, dtype=int)
    period_payoffs = np.zeros((N_paths, N_periods))
    for i in range(N_periods):
        start_index = period_indices[i]
        settlement_index = period_indices[i + 1] - 1
        X_i = np.zeros(N_paths)
        days_in_period = settlement_index - start_index + 1
        for day in range(days_in_period):
            for sim in range(N_paths):
                price = S[start_index + day, sim]
                if price < B:
                    if delta * (price - K) >= 0:
                        X_i[sim] += m
                    else:
                        X_i[sim] += G * m
                else:
                    X_i[sim] += 0
        S_settlement = S[settlement_index, :]
        A_i = delta * (S_settlement - K)
        period_payoff = X_i * A_i
        t_settlement = settlement_index / days_per_year
        #discount_factor = np.exp(-r * t_settlement)
        discount_factor = 0.995
        period_payoffs[:, i] = period_payoff * discount_factor
    total_payoffs = np.sum(period_payoffs, axis=1)
    price = np.mean(total_payoffs)
    return price
    
if __name__ == '__main__':
    S0 = 214.945
    r = 0.06
    sigma = 0.0
    T = 1/12
    days_per_year = 252
    N_periods = 1
    N_paths = 10000
    m = 1.0
    K = 214.945
    B = 1000.0
    G = 1
    contract_type = 'accumulator'
    
    price = simulate_acdc(S0, r, sigma, T, N_paths, N_periods, days_per_year, m, K, B, G, contract=contract_type, seed=100)
    print(f"Estimated price of the {contract_type} ACDC contract: {price:.4f}")




Estimated price of the accumulator ACDC contract: 21.4355


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def simulate_path_milstein(s0, mu, sigma, horizon, timesteps, n_sims, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    dt = horizon / timesteps  # Time step
    S = np.zeros((timesteps, n_sims))  # Initialize stock price matrix
    S[0, :] = s0  # Set initial stock price
    
    for i in range(1, timesteps):
        w = np.random.standard_normal(n_sims)  # Standard normal random variable
        dW = np.sqrt(dt) * w  # Brownian motion increment
        
        # Milstein correction term
        correction = 0.5 * sigma**2 * S[i-1, :] * (dW**2 - dt)
        
        # Apply the Milstein update
        S[i, :] = S[i-1, :] + mu * S[i-1, :] * dt + sigma * S[i-1, :] * dW + correction
    
    return S

def simulate_acdc(S0, r, sigma, T, N_paths, N_periods, days_per_year, m, K, B, G, contract='accumulator', seed=None):
    n_steps = int(days_per_year * T)  # Total simulation steps; for 1 year, n_steps = 252
    S = simulate_path_milstein(S0, r, sigma, T, n_steps, N_paths, seed)
    delta = 1 if contract.lower() == 'accumulator' else -1
    period_indices = np.linspace(0, n_steps, N_periods + 1, dtype=int)
    period_payoffs = np.zeros((N_paths, N_periods))
    
    for i in range(N_periods):
        start_index = period_indices[i]
        # For the final period, use the full period so that t_settlement = n_steps/days_per_year
        if i == N_periods - 1:
            settlement_index = n_steps - 1  # Last index of S (i.e., day 251, which represents 252 days)
            t_settlement = n_steps / days_per_year  # t_settlement = 252/252 = 1 year
            S_settlement = S[settlement_index, :]
        else:
            settlement_index = period_indices[i + 1] - 1
            t_settlement = (settlement_index + 1) / days_per_year  # +1 to count days correctly
            S_settlement = S[settlement_index, :]
        
        X_i = np.zeros(N_paths)
        days_in_period = settlement_index - start_index + 1
        
        for day in range(days_in_period):
            for sim in range(N_paths):
                price = S[start_index + day, sim]
                if price < B:
                    if delta * (price - K) >= 0:
                        X_i[sim] += m  # m = 1
                    else:
                        X_i[sim] += G * m  # G = 1
                else:
                    X_i[sim] += 0
        
        A_i = delta * (S_settlement - K)
        period_payoff = X_i * A_i
        discount_factor = np.exp(-r * t_settlement)
        period_payoffs[:, i] = period_payoff * discount_factor
        
        # Print discount factor and the average discounted payoff for this period
        avg_payoff = np.mean(period_payoffs[:, i])
        print(f"Period {i+1}: t_settlement = {t_settlement*days_per_year:.3f} days, discount_factor = {discount_factor:.9f}, avg discounted payoff = {avg_payoff:.4f}")
    
    total_payoffs = np.sum(period_payoffs, axis=1)
    price = np.mean(total_payoffs)
    return price

if __name__ == '__main__':
    S0 = 214.945
    r = 0.05841
    sigma = 0.0
    T = 12/12  # 1 year
    days_per_year = 252
    N_periods = 12
    N_paths = 10000
    m = 1.0
    K = 214.945
    B = 216.062714
    G = 1
    contract_type = 'accumulator'
    
    price = simulate_acdc(S0, r, sigma, T, N_paths, N_periods, days_per_year, m, K, B, G, contract=contract_type, seed=100)
    print(f"Estimated price of the {contract_type} ACDC contract: {price:.4f}")


Period 1: t_settlement = 21.000 days, discount_factor = 0.995144327, avg discounted payoff = 20.8692
Period 2: t_settlement = 42.000 days, discount_factor = 0.990312232, avg discounted payoff = 4.0646
Period 3: t_settlement = 63.000 days, discount_factor = 0.985503599, avg discounted payoff = 0.0000
Period 4: t_settlement = 84.000 days, discount_factor = 0.980718316, avg discounted payoff = 0.0000
Period 5: t_settlement = 105.000 days, discount_factor = 0.975956269, avg discounted payoff = 0.0000
Period 6: t_settlement = 126.000 days, discount_factor = 0.971217345, avg discounted payoff = 0.0000
Period 7: t_settlement = 147.000 days, discount_factor = 0.966501431, avg discounted payoff = 0.0000
Period 8: t_settlement = 168.000 days, discount_factor = 0.961808416, avg discounted payoff = 0.0000
Period 9: t_settlement = 189.000 days, discount_factor = 0.957138189, avg discounted payoff = 0.0000
Period 10: t_settlement = 210.000 days, discount_factor = 0.952490639, avg discounted payoff =